<a href="https://colab.research.google.com/github/pydevcasts/Dr.RahimPour_ml_practise/blob/main/PYNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

کد ارائه‌شده یک پیاده‌سازی ساده از شبکه عصبی با استفاده از الگوریتم پس‌انتشار (Back-Propagation) در زبان برنامه‌نویسی پایتون است. این شبکه برای یادگیری تابع XOR طراحی شده است. در ادامه، اجزای اصلی کد و عملکرد آن به‌طور خلاصه توضیح داده می‌شود:

### اجزای اصلی کد:

1. **وارد کردن کتابخانه‌ها**:
   - `math`: برای استفاده از توابع ریاضی.
   - `random`: برای تولید اعداد تصادفی.

2. **توابع کمکی**:
   - `rand(a, b)`: یک عدد تصادفی بین `a` و `b` تولید می‌کند.
   - `makeMatrix(I, J, fill=0.0)`: یک ماتریس دو بعدی با ابعاد `I` در `J` ایجاد می‌کند و تمام عناصر آن را با مقدار `fill` پر می‌کند.
   - `sigmoid(x)`: تابع سیگموئید (tanh) را محاسبه می‌کند.
   - `dsigmoid(y)`: مشتق تابع سیگموئید را محاسبه می‌کند.

3. **کلاس NN**:
   - `__init__(self, ni, nh, no)`: سازنده کلاس که تعداد نودهای ورودی (`ni`)، نودهای مخفی (`nh`) و نودهای خروجی (`no`) را تعریف می‌کند. همچنین وزن‌ها را به صورت تصادفی مقداردهی می‌کند.
   - `update(self, inputs)`: ورودی‌ها را به شبکه می‌دهد و فعالیت‌های نودهای مخفی و خروجی را محاسبه می‌کند.
   - `backPropagate(self, targets, N, M)`: خطا را محاسبه کرده و وزن‌ها را به‌روز می‌کند. `N` نرخ یادگیری و `M` عامل مومنتوم است.
   - `test(self, patterns)`: شبکه را با الگوهای ورودی آزمایش می‌کند.
   - `weights(self)`: وزن‌های ورودی و خروجی را چاپ می‌کند.
   - `train(self, patterns, iterations=1000, N=0.5, M=0.1)`: شبکه را با استفاده از الگوهای ورودی آموزش می‌دهد.

4. **تابع demo()**:
   - الگوهای ورودی و خروجی برای تابع XOR را تعریف می‌کند.
   - یک شبکه عصبی با دو نود ورودی، دو نود مخفی و یک نود خروجی ایجاد می‌کند.
   - شبکه را با الگوهای تعریف‌شده آموزش می‌دهد و سپس نتایج را آزمایش می‌کند.

### نتیجه‌گیری:

این کد یک شبکه عصبی ساده را پیاده‌سازی می‌کند که می‌تواند تابع 
XOR 
را یاد بگیرد. با استفاده از الگوریتم پس‌انتشار، شبکه وزن‌های خود را به‌روز می‌کند تا خطای پیش‌بینی را کاهش دهد.
 این پیاده‌سازی می‌تواند به عنوان یک نقطه شروع برای درک عمیق‌تر شبکه‌های عصبی و یادگیری ماشین استفاده شود.

https://colab.research.google.com/drive/1_-Ez1p73aOEP629zJJHIl4iReIHqqNWr




In [5]:
# Back-Propagation Neural Networks
#
# Written in Python.  See http://www.python.org/
# Placed in the public domain.
# Neil Schemenauer <nas@arctrix.com>

import math
import random
import string

random.seed(0)

# calculate a random number where:  a <= rand < b
def rand(a, b):
    return (b-a)*random.random() + a

# Make a matrix (we could use NumPy to speed this up)
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill]*J)
    return m

# our sigmoid function, tanh is a little nicer than the standard 1/(1+e^-x)
def sigmoid(x):
    return math.tanh(x)

# derivative of our sigmoid function, in terms of the output (i.e. y)
def dsigmoid(y):
    return 1.0 - y**2

class NN:
    def __init__(self, ni, nh, no):
        # number of input, hidden, and output nodes
        self.ni = ni + 1 # +1 for bias node
        self.nh = nh
        self.no = no

        # activations for nodes
        self.ai = [1.0]*self.ni
        self.ah = [1.0]*self.nh
        self.ao = [1.0]*self.no

        # create weights
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        # set them to random vaules
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2.0, 2.0)

        # last change in weights for momentum
        self.ci = makeMatrix(self.ni, self.nh)
        self.co = makeMatrix(self.nh, self.no)

    def update(self, inputs):
        if len(inputs) != self.ni-1:
            raise ValueError('wrong number of inputs')

        # input activations
        for i in range(self.ni-1):
            #self.ai[i] = sigmoid(inputs[i])
            self.ai[i] = inputs[i]

        # hidden activations
        for j in range(self.nh):
            summ = 0.0
            for i in range(self.ni):
                summ = summ + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(summ)

        # output activations
        for k in range(self.no):
            summ = 0.0
            for j in range(self.nh):
                summ = summ + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(summ)

        return self.ao[:]


    def backPropagate(self, targets, N, M):
        if len(targets) != self.no:
            raise ValueError('wrong number of target values')

        # calculate error terms for output
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k]-self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        # calculate error terms for hidden
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        # update output weights
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
                self.co[j][k] = change
                #print N*change, M*self.co[j][k]

        # update input weights
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
                self.ci[i][j] = change

        # calculate error
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5*(targets[k]-self.ao[k])**2
        return error


    def test(self, patterns):
        for p in patterns:
            print(p[0], '->', self.update(p[0]))

    def weights(self):
        print ('Input weights:')
        for i in range(self.ni):
            print (self.wi[i])
        print
        print('Output weights:')
        for j in range(self.nh):
            print(self.wo[j])

    def train(self, patterns, iterations=1000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, N, M)
            if i % 100 == 0:
                pass #print 'error %-14f' % error


def demo():
    # Teach network XOR function
    pat = [
        [[0,0], [0]],
        [[0,1], [1]],
        [[1,0], [1]],
        [[1,1], [0]]
    ]

    # create a network with two input, two hidden, and one output nodes
    n = NN(2, 2, 1)
    # train it with some patterns
    n.train(pat)
    # test it
    n.test(pat)



if __name__ == '__main__':
    demo()

[0, 0] -> [0.00424108155062589]
[0, 1] -> [0.9821508029410748]
[1, 0] -> [0.9820129388618121]
[1, 1] -> [-0.0011469114721422528]
